# Revised case normalization for Winterthur 2019

Notes:

- 90 cases can be matched with the DB (if ignoring patientID), otherwise 57.

In [1]:
import sys
sys.path.insert(0, '/home/jovyan/work')

from src.revised_case_normalization.notebook_functions.global_configs import *
from src.revised_case_normalization.notebook_functions.normalize import normalize
from src.revised_case_normalization.notebook_functions.revise import revise
from src.revised_case_normalization.notebook_functions.group import group
from src.revised_case_normalization.notebook_functions.update_db import update_db

In [2]:
file_info = FileInfo(
         os.path.join(ROOT_DIR, 'raw_data/Winterthur.xlsx'),
         'KSW',
         '2019',
         'Änderungen Winterthur 2019')

print(file_info)



FileInfo(path='/home/jovyan/work/src/revised_case_normalization/raw_data/Winterthur.xlsx', hospital_name_db='KSW', year='2019', sheet='Änderungen Winterthur 2019')


In [4]:

columns_to_rename = dict(COLUMNS_TO_RENAME)
columns_to_rename.pop("admno")
columns_to_rename['fid'] = CASE_ID_COL

revised_cases_df = normalize(file_info, columns_mapper = columns_to_rename)
revised_cases_df
revised_cases_df.dtypes

2022-11-16 11:04:27.811 | INFO     | src.revised_case_normalization.notebook_functions.normalize:normalize:35 - Read 90 cases for KSW 2019
2022-11-16 11:04:27.815 | INFO     | src.revised_case_normalization.notebook_functions.normalize:normalize:70 - TYPES:
tranche                                                 string
datum                                                   string
case_id                                                 object
patient_id                                              object
kkik                                                    string
gender                                                  string
age_years                                                int64
bfs_code                                                string
duration_of_stay                                         int64
pflegetage neu                                          string
pccl                                                     int64
pccl neu                                         

case_id              object
patient_id           object
gender               object
age_years             int64
duration_of_stay      int64
case_id_norm         object
old_pd               object
primary_diagnosis    object
bfs_code             object
pccl                  int64
drg                  object
added_icds           object
removed_icds         object
added_chops          object
removed_chops        object
dtype: object

# Match revised DtoD data with the BfS data from the database

In [6]:
cols_to_join = list(VALIDATION_COLS)
cols_to_join.remove(CASE_ID_COL)
cols_to_join.append(NORM_CASE_ID_COL)
# Ignore the patient ID in this dataset, because it is encrypted
cols_to_join.remove(PATIENT_ID_COL)

revised_cases_df.dtypes


2022-11-16 11:05:26.017 | INFO     | src.service.bfs_cases_db_service:get_sociodemographics_for_hospital_year:90 - Read 27903 rows from the DB, for the hospital 'KSW' in 2019


,aimedic_id,case_id,primary_diagnosis,secondary_diagnoses,primary_procedure,secondary_procedures,gender,age_years,age_days,gestation_age,duration_of_stay,ventilation_hours,grouper_admission_type,admission_date,admission_weight,grouper_discharge_type,discharge_date
0,312427,900584428,K5733,"[D62, D648, E559, R391, F708, ...]",4523::20190417,"[451310::20190417, 990410::20190417, 4523::20190422, 451310::20190422]",M,75,0,0,7,0,01,2019-04-16,0,00,2019-04-23
1,312817,900538395,K8050,"[K8510, K8020, K298, E876, K290]",4516::20190219,"[887415::20190219, 887430::20190219, 5185::20190219, 5188::20190219]",M,55,0,0,5,0,01,2019-02-16,0,00,2019-02-21
2,313807,900530732,J4400,"[M1007, I480, E871, Y579, I1090]",8937::20190206,[],M,68,0,0,6,0,01,2019-02-06,0,00,2019-02-12
3,313898,900656399,C61,"[C772, C795, D62, N390, B965]",6022::20190807,"[399A11:R:20190807, 990410::20190807]",M,78,0,0,3,0,01,2019-08-07,0,00,2019-08-10
4,314039,900531410,J4409,"[J101, J9600, K590, Y579, E039, ...]",998425::20190207,[],M,47,0,0,12,0,01,2019-02-07,0,00,2019-02-19


In [ ]:
if unmatched.shape[0] > 0:
    display(unmatched)

In [ ]:
revisions_update, diagnoses_update, procedures_update = group(revised_cases)
revisions_update[REVISION_DATE_COL] = '2022-12-31'

In [ ]:
revisions_update

In [ ]:
update_db(revisions_update, diagnoses_update, procedures_update)